In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_train_prepro = pd.read_csv('data/preprocessed_train_data_with_date_hol_concat.csv')
df_aircraft_model = pd.read_csv('data/full_aircraft_model_mapping_CORRECTED.csv')
df_passenger = pd.read_csv('data/passenger_data.csv')

In [5]:
test_data = pd.read_csv('data/preprocessed_test_data.csv')

In [6]:
test_data

,Unnamed: 0,id,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,duration
0,0,test_id_0,DJE,TUN,2016-05-04 06:40:00,2016-05-04 07:30:00,ATA,TU 32AIMF,3000.0
1,1,test_id_1,TUN,BKO,2016-05-05 15:20:00,2016-05-05 20:05:00,ATA,TU 320IMW,17100.0
2,2,test_id_2,FRA,TUN,2016-05-06 10:00:00,2016-05-06 12:25:00,ATA,TU 32AIMC,8700.0
3,3,test_id_3,BEY,TUN,2016-05-11 09:40:00,2016-05-11 13:10:00,ATA,TU 31BIMO,12600.0
4,4,test_id_4,ORY,MIR,2016-05-11 09:50:00,2016-05-11 12:35:00,ATA,TU 736IOQ,9900.0
...,...,...,...,...,...,...,...,...,...
9328,9328,test_id_9328,TUN,NCE,2018-09-12 14:15:00,2018-09-12 15:45:00,ATA,TU 320IMV,5400.0
9329,9329,test_id_9329,TUN,TUN,2018-09-27 22:00:00,2018-09-28 01:00:00,SCH,TU 32AIMG,10800.0
9330,9330,test_id_9330,SJJ,TUN,2018-09-03 09:20:00,2018-09-03 11:10:00,SCH,TU CR9ISA,6600.0
9331,9331,test_id_9331,TUN,DJE,2018-09-15 14:30:00,2018-09-15 15:30:00,SCH,UG AT7LBD,3600.0


# Map aircraft model to main df

In [3]:
# Map aircraft_model to df_train_prepro using aircraft_code
df_train_prepro['aircraft_model'] = df_train_prepro['aircraft_code'].map(
    df_aircraft_model.set_index('aircraft_code')['aircraft_model']
)

# Map Passenger data to main df

In [4]:
# Ensure departure_date is datetime
df_train_prepro['departure_date'] = pd.to_datetime(df_train_prepro['departure_date'])

# Extract year from departure_date
df_train_prepro['year'] = df_train_prepro['departure_date'].dt.year

# Melt df_passenger to long format for easy merging
df_passenger_long = df_passenger.melt(
    id_vars=['IATA_Code'],
    value_vars=['2016 Passengers', '2017 Passengers', '2018 Passengers'],
    var_name='year_col',
    value_name='num_passenger_year'
)
df_passenger_long['year'] = df_passenger_long['year_col'].str.extract(r'(\d{4})').astype(int)

# Merge passenger info into flight data (use 'departure_point' instead of 'IATA_Code')
df_train_prepro = df_train_prepro.merge(
    df_passenger_long[['IATA_Code', 'year', 'num_passenger_year']],
    left_on=['departure_point', 'year'],
    right_on=['IATA_Code', 'year'],
    how='left'
)

# Drop the temporary columns if you don't need them
df_train_prepro = df_train_prepro.drop(columns=['year', 'IATA_Code'])

In [5]:
# Clean and convert 'num_passenger_year' to integer
df_train_prepro['num_passenger_year'] = (
    df_train_prepro['num_passenger_year']
    .replace(',', '', regex=True)
    .astype(float)
    .astype('Int64')
)

# Map Distance km data to main df

In [6]:
import airportsdata

# Load the airports database (IATA codes)
airports = airportsdata.load('IATA')

def get_lat_lon(iata):
    info = airports.get(iata)
    if info:
        return pd.Series({'lat': info['lat'], 'lon': info['lon']})
    else:
        return pd.Series({'lat': None, 'lon': None})

# Populate departure coordinates
dep_coords = df_train_prepro['departure_point'].apply(get_lat_lon)
df_train_prepro['dep_lat'] = dep_coords['lat']
df_train_prepro['dep_long'] = dep_coords['lon']

# Populate arrival coordinates
arr_coords = df_train_prepro['arrival_point'].apply(get_lat_lon)
df_train_prepro['arr_lat'] = arr_coords['lat']
df_train_prepro['arr_long'] = arr_coords['lon']

# Fill SXF manually if still missing
sxf_lat, sxf_long = 52.380001, 13.522500
df_train_prepro.loc[df_train_prepro['departure_point'] == 'SXF', ['dep_lat', 'dep_long']] = [sxf_lat, sxf_long]
df_train_prepro.loc[df_train_prepro['arrival_point'] == 'SXF', ['arr_lat', 'arr_long']] = [sxf_lat, sxf_long]

In [7]:
# Haversine formula to calculate distance in km
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Calculate the distance for each row
df_train_prepro['distance_km'] = haversine(
    df_train_prepro['dep_lat'],
    df_train_prepro['dep_long'],
    df_train_prepro['arr_lat'],
    df_train_prepro['arr_long']
)

df_train_prepro['distance_km'] = (df_train_prepro['distance_km'].round(2) * 100).fillna(0).astype(int)

# Map expected duration data to main df

In [8]:
route_durations = df_train_prepro.groupby('route')['duration'].median().reset_index()
route_durations.rename(columns={'duration': 'expected_duration'}, inplace=True)
route_durations.sort_values(by='expected_duration', ascending=False, inplace=True)

In [9]:
# Step 3: Merge back into the original dataframe
df_train_prepro = df_train_prepro.merge(route_durations, on='route', how='left')

In [10]:
df_train_prepro['delay_relative_to_expected'] = df_train_prepro['duration'] - df_train_prepro['expected_duration']
df_train_prepro['duration_ratio'] = df_train_prepro['duration'] / df_train_prepro['expected_duration']

In [11]:
df_train_prepro

,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,dep_hour,dep_day,...,aircraft_model,num_passenger_year,dep_lat,dep_long,arr_lat,arr_long,distance_km,expected_duration,delay_relative_to_expected,duration_ratio
0,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,-19.0,9300.0,4,16,...,Airbus A320,5200000,36.85100,10.22720,41.27533,28.75200,166955,9300.0,0.0,1.000000
1,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,-48.0,10200.0,14,17,...,Boeing 737-600,993429,33.87500,10.77550,47.15320,-1.61073,180522,10200.0,0.0,1.000000
2,TUN,MED,2016-01-20 19:40:00,2016-01-21 00:00:00,ATA,TU 320IMR,-16.0,15600.0,19,20,...,Airbus A320-200,5200000,36.85100,10.22720,24.55340,39.70510,311502,14400.0,1200.0,1.083333
3,IST,TUN,2016-01-21 20:10:00,2016-01-21 23:00:00,ATA,TU 320IMU,-8.0,10200.0,20,21,...,Airbus A320-200,60378393,41.27533,28.75200,36.85100,10.22720,166955,10200.0,0.0,1.000000
4,CMN,TUN,2016-01-22 17:45:00,2016-01-22 20:10:00,ATA,TU 320IMR,-37.0,8700.0,17,22,...,Airbus A320-200,8244481,33.36750,-7.58997,36.85100,10.22720,166364,8700.0,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99742,TUN,DJE,2018-04-18 08:20:00,2018-04-18 09:10:00,ATA,TU 31BIMQ,8.0,3000.0,8,18,...,Airbus A318,6000000,36.85100,10.22720,33.87500,10.77550,33463,3600.0,-600.0,0.833333
99743,ORY,DJE,2018-12-05 10:15:00,2018-12-05 13:05:00,ATA,TU 736IOL,20.0,10200.0,10,5,...,Boeing 737-600,33120685,48.72530,2.35944,33.87500,10.77550,179204,10200.0,0.0,1.000000
99744,BRU,DJE,2018-12-05 09:45:00,2018-12-05 12:50:00,ATA,TU 736IOR,15.0,11100.0,9,5,...,Boeing 737-600,25700000,50.90140,4.48444,33.87500,10.77550,196069,11100.0,0.0,1.000000
99745,ORY,TUN,2018-12-04 18:30:00,2018-12-04 20:55:00,ATA,TU 32AIMI,22.0,8700.0,18,4,...,Airbus A320,33120685,48.72530,2.35944,36.85100,10.22720,146628,8700.0,0.0,1.000000


In [12]:
df_train_prepro.to_csv('data/data_all_col_upto_duration_ratio.csv', index=False)